In [1]:
import boll

In [64]:
import importlib
importlib.reload(boll)

<module 'boll' from 'c:\\ubion_2025\\python\\250324\\boll.py'>

In [2]:
import yfinance as yf

In [45]:
from datetime import datetime

In [19]:
# 티커를 기준으로 데이털 로드 
ticker_aapl = yf.Ticker('BTC-USD')

In [20]:
df_aapl = ticker_aapl.history(
    start = '2010-01-01'
)

In [50]:
type(df_aapl.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [65]:
band_aapl = boll.create_band(df_aapl, _col='Close', _start='2010-01-01')

In [48]:
start = '2015-01-01'

In [46]:
start = datetime.strptime(start, '%Y-%m-%d')

In [49]:
band_aapl[start:]

,Close,center,up,down
Date,,,,
2015-01-01 00:00:00+00:00,314.248993,324.905199,349.013769,300.796628
2015-01-02 00:00:00+00:00,315.032013,323.287999,345.297786,301.278212
2015-01-03 00:00:00+00:00,281.082001,319.760500,345.017684,294.503316
2015-01-04 00:00:00+00:00,264.195007,315.703000,348.578640,282.827360
2015-01-05 00:00:00+00:00,274.473999,313.073599,350.254140,275.893058
...,...,...,...,...
2025-03-20 00:00:00+00:00,84167.195312,85127.125391,92430.525040,77823.725741
2025-03-21 00:00:00+00:00,84043.242188,85027.691797,92333.374373,77722.009221
2025-03-22 00:00:00+00:00,83832.484375,84506.898438,90391.851236,78621.945639
